# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Dropout, Conv2D, Dense, MaxPooling2D, AveragePooling2D, Flatten, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import random

# Loading the Data

In [ ]:
df = pd.read_csv('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')
df.head()

In [ ]:
train_dir = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/'
test_dir = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'

# Show some labels and images

for i in range(6):
    plt.subplot(2,3,i+1)
    img = cv2.imread(train_dir+df.loc[i,'Image'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(df.loc[i,'Class'])
    plt.axis("off")
plt.show()

# Preprocess the images and split into train and validation

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,
                            validation_split=0.2)
                            #height_shift_range=0.2,
                            #shear_range=0.2,
                            #horizontal_flip=True,
                            #width_shift_range=0.2,

In [ ]:
train_gen = datagen.flow_from_dataframe(df,
                                       directory=train_dir,
                                       x_col='Image',
                                       y_col='Class',
                                       color_mode='rgb',
                                       target_size=(331,331),
                                       class_mode='categorical',
                                       shuffle=True,
                                       subset='training',
                                       batch_size=128)

val_gen = datagen.flow_from_dataframe(df,
                                    directory=train_dir,
                                    x_col='Image',
                                    y_col='Class',
                                    color_mode='rgb',
                                    target_size=(331,331),
                                    class_mode='categorical',
                                    shuffle=True,
                                    subset='validation',
                                    batch_size=128) 

In [ ]:
train_gen.class_indices

# Create and train the model

In [ ]:
base_layer = NASNetLarge(pooling='avg', weights='imagenet', include_top=True)

model = Sequential([base_layer,
                   Dense(units = 512, activation='relu'),
                   Dropout(0.4),
                   Dense(units = 256, activation='relu'),
                   Dropout(0.3),
                   Dense(units=6, activation='softmax')])

model.layers[0].trainable=False

opt = Adam(lr=1e-3, decay=1e-5)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

# Evaluate on validation data

In [ ]:
results = model.evaluate(val_gen)
print("val_loss, val_acc: ", results)

# Training graphs

In [ ]:
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['accuracy'], label='Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(borderpad=2)
plt.show()
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['loss'], label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(borderpad=2)
plt.show()

In [ ]:
def preprocess(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (331, 331))
    img = img/255.
    return img

In [ ]:
old_labels = train_gen.class_indices
NUM_TO_LABEL = dict((v,k) for k,v in old_labels.items())
NUM_TO_LABEL

# Show an image and label

In [ ]:
path = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/image1014.jpg'
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
img = preprocess(path)
plt.title(NUM_TO_LABEL[model.predict(img.reshape(1,331,331,3)).argmax()])
plt.axis("off")
plt.show()

# Create the test predictions

In [ ]:
files = os.listdir(test_dir)

y_pred = []

for file in files:
    img = preprocess(test_dir+file)
    y_pred.append(NUM_TO_LABEL[model.predict(img.reshape(1,331,331,3)).argmax()])

In [ ]:
dict = {'Image': files, 'Class': y_pred}

pf = pd.DataFrame(dict)
pf.to_csv('results.csv')